In [4]:
import numpy as np
from scipy import spatial
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

In [3]:
# You do not need to load both embeddings! For test runs, simply load the small embedding. It is much faster!
embeddings_dict = {}

In [2]:
# load big embedding into dictionary
# embedding from http://vectors.nlpl.eu/repository/
# English CoNLL17 corpus
# This takes a while!

embeddings_dict_big = {}

counter = 0
errors = []
embeddings_dict = {}
with open("/Users/floriannolte/Downloads/40/model.txt", 'r', encoding='ISO-8859-1') as f: # encoding needed for special characters
    for line in f:
        counter += 1
        values = line.split()
        word = values[0]
        try:
            vector = np.asarray(values[1:], "float32")
            embeddings_dict_big[word] = vector
        except:
            errors.append((counter, word)) # save words where the program breaks for some reason

# delete weird entries that have wrong dimensionality for some reason
liste = []
for k, v in embeddings_dict_big.items():
    if np.shape(v)[0] != 100:
        liste.append(k)

for w in liste:
    del embeddings_dict_big[w]
    
# del embeddings_dict["4027169"] # first line is weird

In [18]:
# ALTERNATIVELY: load smol embedding into dictionary
# this is the glove embedding from the authors of the paper
# corpus: wikipedia 2014 + gigaword
# https://nlp.stanford.edu/projects/glove/
embeddings_dict_smol = {}

with open("glove.6B.100d.txt", 'r') as f: 
    for line in f:
        counter += 1
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict_smol[word] = vector

In [ ]:
# Choose embedding to work with

embeddings_dict = embeddings_dict_smol
#embeddings_dict = embeddings_dict_big

In [73]:
# 2 x 50 random words to compare the companies against.
random_words_1 = "announcement friend ask pan tail quiet swarm calm glance level tolerate my part devote abortion memorial chaos bleed trench ensure mask create indulge salesperson conservative pier speculate genetic fiction regulation young harvest responsible beach squeeze seek field lip seasonal account complain angel dawn transport observation budge council sunshine farewell slide".split(" ")
random_words_2 = "healthy rage grain surprise trustee prison diagram promise shift escape amputate gallery note stress policeman edge foundation folk squeeze deck misery partnership distinct title hostage rate conspiracy sandwich abridge night old flourish crack angel example fibre urine courtship tragedy prove reform bush breakdown huge technology strip treat meal dignity wife".split(" ")
random_words = random_words_1 + random_words_2

### Helper functions

In [19]:
# function for finding closest words to one embedding
# warning : very slow!

def find_closest_embeddings(embedding):
    return sorted(embeddings_dict.keys(), key=lambda word: spatial.distance.cosine(embeddings_dict[word], embedding))

In [20]:
# returns angle between two vectors / lists / arrays
def angle_vec(vec1, vec2): 
    dot_product = np.dot(
        np.array(vec1) / np.linalg.norm(np.array(vec1)),
        np.array(vec2) / np.linalg.norm(np.array(vec2))
    )
    return np.degrees(np.arccos(dot_product))

### Companies, frameworks and sentiment-words

In [21]:
companies = ["Google", "Microsoft", "amazon", "Facebook",
             "Intel", "Nvidia", "AMD", "Deepmind", "IBM",
             "Boston", "Dynamics", "Softbank ", "Robotics",
             "PAL", " Robotics", "Willow", "Garage",
             "Alibaba", " Cloud", "H20.ai", "Bosch",
             "SPHERO", "ANYBOTS", "sarcos", "BARRETT",
             "PETRONICS", "dronesense", "embodied",
             "energid" , "irobot", "myomo", "vecna"]

companies = [name.lower().replace(" ","") for name in companies]


frameworks = ["Sci-kit", "Tensorflow", "Theano", "Caffe", "Keras", "Torch", "MicrosoftCNTK",
             "OpenCV", "ROS", "OpenNN", "MxNet", "Amazon", "Matlab", "Accord.NET",
             "Spark","Mllib", "MLPack", "Apche", "Mahout", "Firebase", "Singa",
             "Azure", "Google", "H20", "YARP", "MRPT", "Gazebo",
             "OROCOS", "Spacy", "Facebook", "Gensim"]

frameworks = [name.lower().replace(" ","") for name in frameworks]

In [22]:
# view which companies are not in the embedding

for i in companies:
    try:
        embeddings_dict[i]
        print('%-15s%-15s' % (i, "True"))
    except:
        print('%-15s%-15s' % (i, "False"))
    # look at "printf-style String Formatting" in https://docs.python.org/3/library/stdtypes.html for info about nice formatting

google         True           
microsoft      True           
amazon         True           
facebook       True           
intel          True           
nvidia         True           
amd            True           
deepmind       False          
ibm            True           
boston         True           
dynamics       True           
softbank       True           
robotics       True           
pal            True           
robotics       True           
willow         True           
garage         True           
alibaba        True           
cloud          True           
h20.ai         False          
bosch          True           
sphero         False          
anybots        False          
sarcos         True           
barrett        True           
petronics      False          
dronesense     False          
embodied       True           
energid        False          
irobot         True           
myomo          False          
vecna          True           


In [23]:
# view which frameworks are not in the embedding

for i in frameworks:
    try:
        embeddings_dict[i]
        print('%-15s%-15s' % (i, "True"))
    except:
        print('%-15s%-15s' % (i, "False"))

sci-kit        False          
tensorflow     False          
theano         True           
caffe          True           
keras          True           
torch          True           
microsoftcntk  False          
opencv         False          
ros            True           
opennn         False          
mxnet          False          
amazon         True           
matlab         True           
accord.net     False          
spark          True           
mllib          False          
mlpack         False          
apche          False          
mahout         True           
firebase       True           
singa          True           
azure          True           
google         True           
h20            True           
yarp           False          
mrpt           False          
gazebo         True           
orocos         False          
spacy          True           
facebook       True           
gensim         False          


In [24]:
# simple positive and negative words

positive_functional = "secure, safe, robust, reliable, trustworthy, efficient, interactive, user-friendly, maintained, stable, outperform, convenient, intelligent, automatic, automated".split(", ")
negative_functional = "insecure, breach, slow, complicated, unstable".split(", ")
positive_society = "famous, open-source, fair, ethical".split(", ")
negative_society = "criticism, illegal, crime, fake, violate, unethical, biased, offensive".split(", ")

# get embeddings for sentiment words
p_f_embed = []
for word in positive_functional:
    p_f_embed.append(embeddings_dict[word])

p_s_embed = []
for word in positive_society:
    p_s_embed.append(embeddings_dict[word])
    
n_f_embed = []
for word in negative_functional:
    n_f_embed.append(embeddings_dict[word])
    
n_s_embed = []
for word in negative_society:
    n_s_embed.append(embeddings_dict[word])

In [25]:
# sentiment vectors from word pairs

functional = [
    ("secure","insecure"),
    ("safe","unsafe"),
    ("reliable","unreliable"),
    ("complicated","simple"),
    ("fast","slow"),
    ("stable","unstable"),
    ("intelligent","dumb"),
    ("convenient","inconvenient")
]
society = [
    ("good","bad"),
    ("famous","unknown"),
    ("open-source","locked"),
    ("fair","biased"),
    ("legal","illegal"),
    ("ethical","unethical"),
    ("violate","conform"),
    ("criticism","praise")
]

# get embeddings for sentiment words
functional_embed = []
for pair in functional:
    functional_embed.append(embeddings_dict[pair[1]] - embeddings_dict[pair[0]])

society_embed = []
for pair in society:
    society_embed.append(embeddings_dict[pair[1]] - embeddings_dict[pair[0]])

### 1st approach: Distances between companies and sentiment words

In [26]:
# for every element, returns average distance to the words in the list (as list)
def avg_dist(elements, words):
    output = []
    for elem in elements:
        dist = 0
        try:
            for word in words:
                dist += spatial.distance.cosine(embeddings_dict[elem], embeddings_dict[word])
            dist = dist / len(words)
        except: # elem not in vocabulary : do nothing
            pass
        output.append([elem, dist])
    return output

In [27]:
# print average distance to sentiment words (0.0 = not in dataset)

p_f_comp = avg_dist(companies, positive_functional)
n_f_comp = avg_dist(companies, negative_functional)
p_s_comp = avg_dist(companies, positive_society)
n_s_comp = avg_dist(companies, negative_society)

for c in range(len(companies)):
    print("%-10s pos avg: %-10.6s neg avg: %-10.6s" % (p_f_comp[c][0], (p_f_comp[c][1] + p_s_comp[c][1])/2 , (n_f_comp[c][1] + n_s_comp[c][1])/2))
    # % = value, - = left align, 10 = distance or position, .6 = precision, s = converting to string

google     pos avg: 0.7634     neg avg: 0.8270    
microsoft  pos avg: 0.7375     neg avg: 0.7713    
amazon     pos avg: 0.8558     neg avg: 0.8590    
facebook   pos avg: 0.8140     neg avg: 0.8442    
intel      pos avg: 0.8235     neg avg: 0.8766    
nvidia     pos avg: 0.9546     neg avg: 1.0255    
amd        pos avg: 0.9065     neg avg: 0.9253    
deepmind   pos avg: 0.0        neg avg: 0.0       
ibm        pos avg: 0.7966     neg avg: 0.9035    
boston     pos avg: 0.8360     neg avg: 0.8890    
dynamics   pos avg: 0.8506     neg avg: 0.8327    
softbank   pos avg: 0.9690     neg avg: 1.0173    
robotics   pos avg: 0.8757     neg avg: 0.9630    
pal        pos avg: 0.9701     neg avg: 0.9548    
robotics   pos avg: 0.8757     neg avg: 0.9630    
willow     pos avg: 0.9742     neg avg: 1.0101    
garage     pos avg: 0.8365     neg avg: 0.9144    
alibaba    pos avg: 1.0228     neg avg: 1.0450    
cloud      pos avg: 0.8556     neg avg: 0.8225    
h20.ai     pos avg: 0.0        

### 2nd approach: Sentiment direction through PCA

In [40]:
# perform pca to get directions of greatest variance = sentiment vectors
# PCA from simple word vectors (first PC of embeddings of functional / societal words)

pca = PCA(n_components=1)
principal_components = pca.fit(p_f_embed + n_f_embed)
functional_sentiment_direction = principal_components.components_[0]

principal_components = pca.fit(p_s_embed + n_s_embed)
society_sentiment_direction = principal_components.components_[0]

In [47]:
# print and save all words with their angles to society words and functional words
# angles around 90° are good.

angles_soc_func = []

print("%-15s%-10s%10s" % ("", "Functional", "Societal"))
for word in companies + frameworks:
    try:
        angles_soc_func.append((word, angle_vec(embeddings_dict[word], functional_sentiment_direction), angle_vec(embeddings_dict[word], society_sentiment_direction)))
        print("%-15s %-12.5s %-12.5s" % (angles_soc_func[-1]))
    except:
        angles_soc_func.append((word, None, None))
        print(word)

               Functional  Societal
google          103.5        90.72       
microsoft       96.57        87.68       
amazon          94.08        94.40       
facebook        102.5        91.26       
intel           91.03        87.24       
nvidia          105.6        80.10       
amd             93.58        80.45       
deepmind
ibm             97.80        86.24       
boston          87.34        110.5       
dynamics        92.45        83.45       
softbank        88.38        85.39       
robotics        106.3        73.58       
pal             87.62        99.04       
robotics        106.3        73.58       
willow          87.56        86.49       
garage          99.49        99.95       
alibaba         97.14        81.96       
cloud           89.89        96.56       
h20.ai
bosch           100.2        91.61       
sphero
anybots
sarcos          98.33        75.15       
barrett         87.09        92.83       
petronics
dronesense
embodied        93.00        7

In [48]:
# code to save values for reference, Dirty Hack
values = []
for word in companies + frameworks:
    try:
        values.append((word, angle_vec(embeddings_dict[word], functional_sentiment_direction), angle_vec(embeddings_dict[word], society_sentiment_direction)))
    except:
        values.append((word, None, None))

for i in values:
    pass#print(i)

In [6]:
# output of the above code, using the big embedding
data_40 = [('google', 81.58132197111395, 85.38306064697701) ,
('microsoft', 78.36052999867374, 77.0228811916095) ,
('amazon', 85.28186932924604, 85.933387302177) ,
('facebook', 88.9508847830484, 94.82193448350525) ,
('intel', 94.08120556760542, 78.0237077154805) ,
('nvidia', 90.33171314960701, 78.04591544990853) ,
('amd', 95.76111446345796, 79.08873679959837) ,
('deepmind', 78.92806778865676, 77.36679111129862) ,
('ibm', 85.45635063356441, 72.9587140422921) ,
('boston', 83.06760921028163, 94.60454058870201) ,
('dynamics', 80.47870836776725, 86.08774306505475) ,
('softbank', 84.02908052690933, 86.31367386642376) ,
('robotics', 72.8452740120154, 77.73747258185175) ,
('pal', 92.52405433992357, 87.40972688031428) ,
('robotics', 72.8452740120154, 77.73747258185175) ,
('willow', 92.7878038847685, 86.38526571500144) ,
('garage', 83.87941769241495, 87.08019065492876) ,
('alibaba', 89.0550431573287, 82.9957963645851) ,
('cloud', 83.55522729406717, 74.80600440144579) ,
('h20.ai', None, None) ,
('bosch', 76.4877570463206, 81.03077193341183) ,
('sphero', 72.51901639439613, 84.38930002249519) ,
('anybots', 75.6938960914384, 70.93412811473013) ,
('sarcos', 82.00368535641135, 75.24767581781794) ,
('barrett', 91.09864319928245, 98.02343856986285) ,
('petronics', None, None) ,
('dronesense', None, None) ,
('embodied', 89.79062034052514, 91.91506877285539) ,
('energid', 81.39549770473579, 82.52686880880094) ,
('irobot', 77.07486632081178, 86.59895044418644) ,
('myomo', 79.51423967820855, 82.5475819227478) ,
('vecna', 88.31910159971396, 84.05769716800604) ,
('sci-kit', None, None) ,
('tensorflow', 74.35898851036781, 65.92541033632993) ,
('theano', 90.68707080148246, 68.46808992119101) ,
('caffe', 81.23021449824734, 75.13977778039687) ,
('keras', 79.23473723063465, 77.70228314026521) ,
('torch', 78.74085813471446, 88.22241981047789) ,
('microsoftcntk', None, None) ,
('opencv', 77.94834113062669, 68.70011051763392) ,
('ros', 98.11706339713135, 84.5791374540656) ,
('opennn', 78.47516401181447, 74.83088873563786) ,
('mxnet', None, None) ,
('amazon', 85.28186932924604, 85.933387302177) ,
('matlab', 77.76507167583912, 70.68675064204588) ,
('accord.net', 76.014251989394, 73.09254826313085) ,
('spark', 82.49301733833002, 93.15978921747266) ,
('mllib', 74.23613598685112, 71.6953677491158) ,
('mlpack', 82.0229633843731, 75.50560541659044) ,
('apche', 85.77292370539041, 79.03605900575565) ,
('mahout', 93.71051699125363, 76.08611873701652) ,
('firebase', 86.77727029960036, 89.59668497799792) ,
('singa', 86.71639641299284, 79.83877368949364) ,
('azure', 78.23977991962062, 76.18484223521166) ,
('google', 81.58132197111395, 85.38306064697701) ,
('h20', 86.29645020702574, 81.54305714291549) ,
('yarp', 81.74074656605538, 79.96225260628653) ,
('mrpt', 74.81719981856898, 75.36126701799999) ,
('gazebo', 87.82167923735082, 80.21430713443701) ,
('orocos', 82.30713731088018, 69.75631515387113) ,
('spacy', 86.51560971466118, 87.6341989870835) ,
('facebook', 88.9508847830484, 94.82193448350525) ,
('gensim', 81.0125357525128, 72.65438778336181)]

In [49]:
# PCA from vectors of word pairs
# first PC of vectors of word pairs

pca2 = PCA(n_components=1)

principal_components2 = pca2.fit(functional_embed)
functional_sentiment_direction2 = principal_components2.components_[0]

principal_components2 = pca2.fit(society_embed)
society_sentiment_direction2 = principal_components2.components_[0]

In [51]:
# print and save all words with their angles to society words and functional words
# angles around 90° are good.

angles_soc_func2 = []

print("%-15s%-10s%10s" % ("", "Functional", "Societal"))
for word in companies + frameworks:
    try:
        angles_soc_func2.append((word, angle_vec(embeddings_dict[word], functional_sentiment_direction2), angle_vec(embeddings_dict[word], society_sentiment_direction2)))
        print("%-15s %-12.5s %-12.5s" % angles_soc_func2[-1])
    except:
        angles_soc_func2.append((word, None, None))
        print(word)

               Functional  Societal
google          82.82        92.48       
microsoft       82.27        89.67       
amazon          89.52        95.36       
facebook        85.79        93.52       
intel           83.55        91.79       
nvidia          94.19        107.4       
amd             91.14        100.0       
deepmind
ibm             80.27        90.40       
boston          73.47        73.80       
dynamics        97.16        86.67       
softbank        93.96        96.05       
robotics        97.30        103.0       
pal             89.35        84.90       
robotics        97.30        103.0       
willow          86.68        92.71       
garage          72.96        78.45       
alibaba         96.30        95.98       
cloud           88.42        81.47       
h20.ai
bosch           88.70        86.77       
sphero
anybots
sarcos          94.44        94.74       
barrett         86.96        90.70       
petronics
dronesense
embodied        92.55        9

In [77]:
# print and save all random words with their angles to society words and functional words
# angles around 90° are good.

angles_soc_func_random = []

#print("%-15s%-10s%10s" % ("", "Functional", "Societal"))
for word in random_words:
    angles_soc_func_random.append((word, angle_vec(embeddings_dict[word], functional_sentiment_direction2), angle_vec(embeddings_dict[word], society_sentiment_direction2)))
    #print("%-15s %-12.5s %-12.5s" % angles_soc_func_random[-1])

In [183]:
# code to save values for reference (dirty hack to avoid saving and loading files)
values2 = []
for word in companies + frameworks:
    try:
        values2.append((word, angle_vec(embeddings_dict[word], functional_sentiment_direction2), angle_vec(embeddings_dict[word], society_sentiment_direction2)))
    except:
        values2.append((word, None, None))

for i in values2:
    pass#print(i, ",")

In [8]:
data_40_2 = [('google', 87.80729601569945, 101.2616002084186) ,
('microsoft', 92.22947565432817, 110.07164865347103) ,
('amazon', 84.22298519162408, 98.37073957342788) ,
('facebook', 91.19328028084082, 91.51849020253316) ,
('intel', 94.42998450709902, 103.26136216765518) ,
('nvidia', 93.46834033729112, 103.31779371734459) ,
('amd', 96.12119166958179, 102.65396343584038) ,
('deepmind', 93.21084544177033, 106.0669040520475) ,
('ibm', 94.03396988383408, 107.96697749573657) ,
('boston', 85.78426620602532, 84.25549514851684) ,
('dynamics', 101.48977662302497, 99.79254005180135) ,
('softbank', 85.47946255530569, 96.38697521969269) ,
('robotics', 91.68728435086771, 105.52983858549132) ,
('pal', 89.9367749521248, 92.88737420651849) ,
('robotics', 91.68728435086771, 105.52983858549132) ,
('willow', 88.20087430511555, 88.78103146006853) ,
('garage', 84.01158632758941, 78.51120196992503) ,
('alibaba', 86.58470898717835, 91.20183815662308) ,
('cloud', 94.23950569972763, 102.21021603083739) ,
('h20.ai', None, None) ,
('bosch', 86.13300389051822, 91.6526205537132) ,
('sphero', 78.37479529498589, 94.67425651443624) ,
('anybots', 93.58907725450175, 107.57486805809884) ,
('sarcos', 93.12075022620122, 102.3142291626249) ,
('barrett', 90.83283905628821, 82.86276948744549) ,
('petronics', None, None) ,
('dronesense', None, None) ,
('embodied', 105.61961368407982, 95.21941810702579) ,
('energid', 92.05817894014919, 101.3757289172342) ,
('irobot', 90.21114339159243, 93.85870258248171) ,
('myomo', 86.759551734634, 100.41563664270775) ,
('vecna', 94.47565428101133, 93.75927845084506) ,
('sci-kit', None, None) ,
('tensorflow', 91.32465113521452, 116.59238909086451) ,
('theano', 94.5420832646147, 106.78243963831068) ,
('caffe', 80.3433866463473, 95.21042232629418) ,
('keras', 85.64530001808315, 98.65104151898463) ,
('torch', 78.73021722446897, 88.20401403330786) ,
('microsoftcntk', None, None) ,
('opencv', 86.4819957335614, 111.75579056042733) ,
('ros', 98.6049338762749, 98.51974202542617) ,
('opennn', 93.20902951597955, 107.82866292336239) ,
('mxnet', None, None) ,
('amazon', 84.22298519162408, 98.37073957342788) ,
('matlab', 89.77010404797144, 115.21627303053977) ,
('accord.net', 87.9976172592865, 111.42730847075244) ,
('spark', 92.82180187353768, 89.45442769250474) ,
('mllib', 88.78843620486964, 109.96137941920102) ,
('mlpack', 88.26684272522562, 107.59817568289475) ,
('apche', 88.87912163757164, 101.82474544321359) ,
('mahout', 98.75995334973769, 104.13602937843352) ,
('firebase', 90.76320736267996, 95.3263933014051) ,
('singa', 91.57197742189429, 101.73901998597614) ,
('azure', 88.16905290339425, 97.54895918444824) ,
('google', 87.80729601569945, 101.2616002084186) ,
('h20', 87.21967183950501, 96.55250398915561) ,
('yarp', 89.09333457641748, 105.49539115557148) ,
('mrpt', 89.2149421989122, 110.31942666080192) ,
('gazebo', 78.12644521584215, 83.15024196561349) ,
('orocos', 93.09475342441513, 109.4087414431444) ,
('spacy', 91.52630880672592, 89.23446259845957) ,
('facebook', 91.19328028084082, 91.51849020253316) ,
('gensim', 96.06760896091329, 112.06899783278402)]

In [133]:
print(angle_vec(embeddings_dict["war"], society_sentiment_direction2))
print(angle_vec(embeddings_dict["bad"], society_sentiment_direction2))

87.32351279397706
90.72444551618975


In [170]:
angle_vec(embeddings_dict["king"] - embeddings_dict["queen"], embeddings_dict["secretary"] - embeddings_dict["coach"])

96.1801

In [159]:
find_closest_embeddings(embeddings_dict["dachshund"])

KeyboardInterrupt: 

In [75]:
# average absolute value of the distance between 90 degrees and the angle of the company vectors to the "society" direction
# measure of how biased the embedding is (between 0 and 90)

avg_societal_bias = 0
avg_functional_bias = 0
counter = 0

for line in angles_soc_func_random:
    angle_soc = line[2]
    angle_func = line[1]
    
    if angle_soc: # check if there is data for the company / framework
        counter += 1
        avg_societal_bias += abs(90 - angle_soc)
        avg_functional_bias += abs(90 - angle_func)
    
avg_functional_bias /= counter
avg_societal_bias /= counter

print("functional bias: ", avg_functional_bias, "societal bias: " , avg_societal_bias)
# CoNLL17: functional bias:  3.9495742106369267 societal bias:  11.044453649519795
# GloVe: functional bias:  5.988432694507943 societal bias:  7.173770563704972
# Random: functional bias:  9.304432622842576 societal bias:  10.992649064172603

functional bias:  9.304432622842576 societal bias:  10.992649064172603
